In [1]:
import sys
from astroquery.gaia import Gaia
from gala.coordinates import GD1Koposov10

sys.path.append('../student_download')
from episode_functions import *

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


# Episode 2: FITS

In [2]:
phi1_min = -55 * u.degree 
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

In [3]:
phi1_rect, phi2_rect = make_rectangle(
    phi1_min, phi1_max, phi2_min, phi2_max)

In [4]:
gd1_frame = GD1Koposov10()
corners = SkyCoord(phi1=phi1_rect, phi2=phi2_rect, frame=gd1_frame)

In [5]:
corners_icrs = corners.transform_to('icrs')

In [6]:
sky_point_list = skycoord_to_string(corners_icrs)

In [7]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

In [8]:
polygon_query_base = """SELECT
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({sky_point_list}))
"""

In [9]:
polygon_query = polygon_query_base.format(columns=columns, 
                          sky_point_list=sky_point_list)

In [10]:
polygon_job = Gaia.launch_job_async(polygon_query)
polygon_results = polygon_job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [11]:
filename = 'gd1_results.fits'
polygon_results.write(filename, overwrite=True)

# Episode 3: HDF5

In [12]:
import pandas as pd
from gala.coordinates import reflex_correct

In [13]:
results_df = make_dataframe(polygon_results)

In [14]:
phi2_min = -1.0 * u.degree
phi2_max = 1.0 * u.degree

In [15]:
phi2 = results_df['phi2']
mask = (phi2 > phi2_min) & (phi2 < phi2_max)

In [16]:
centerline_df = results_df[mask]

In [17]:
pm1_min = -8.9
pm1_max = -6.9
pm2_min = -2.2
pm2_max =  1.0

In [18]:
pm1 = results_df['pm_phi1']
pm2 = results_df['pm_phi2']

pm_mask = (between(pm1, pm1_min, pm1_max) & 
           between(pm2, pm2_min, pm2_max))

In [19]:
selected_df = results_df[pm_mask]

In [20]:
filename = 'gd1_data.hdf'

selected_df.to_hdf(filename, 'selected_df', mode='w')
centerline_df.to_hdf(filename, 'centerline_df')

# Chapter 4: HDF5 cont

In [21]:
from scipy.spatial import ConvexHull
import numpy as np

In [22]:
points = selected_df[['pmra','pmdec']].to_numpy()

In [23]:
hull = ConvexHull(points)

In [24]:
pm_vertices = points[hull.vertices]

In [25]:
phi1_min = -70 * u.degree
phi1_max = -20 * u.degree
phi2_min = -5 * u.degree
phi2_max = 5 * u.degree

In [26]:
phi1_rect, phi2_rect = make_rectangle(
    phi1_min, phi1_max, phi2_min, phi2_max)

In [27]:
corners = SkyCoord(phi1=phi1_rect, 
                   phi2=phi2_rect, 
                   frame=gd1_frame)

corners_icrs = corners.transform_to('icrs')

In [28]:
sky_point_list = skycoord_to_string(corners_icrs)

In [29]:
s = np.array2string(pm_vertices.flatten(), 
                    max_line_width=1000,
                    separator=',')

In [30]:
pm_point_list = s.strip('[]')

In [31]:
d = dict(sky_point_list=sky_point_list, pm_point_list=pm_point_list)

In [32]:
point_series = pd.Series(d)

In [33]:
filename = 'gd1_data.hdf'
point_series.to_hdf(filename, 'point_series')

# Episode 5: CSV

In [34]:
column_list = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.best_neighbour_multiplicity',
               'best.number_of_mates',
               'ps.g_mean_psf_mag',
               'ps.i_mean_psf_mag']

In [35]:
candidate_join_query_base = """
SELECT 
{columns}
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(gaia.ra, gaia.dec), 
                   POLYGON({sky_point_list}))
  AND 1 = CONTAINS(POINT(gaia.pmra, gaia.pmdec),
                   POLYGON({pm_point_list}))
"""

columns = ', '.join(column_list)

candidate_join_query = candidate_join_query_base.format(columns=columns,
                            sky_point_list=point_series['sky_point_list'],
                            pm_point_list=point_series['pm_point_list'])
print(candidate_join_query)


candidate_join_job = Gaia.launch_job_async(query=candidate_join_query)
candidate_table = candidate_join_job.get_results()


SELECT 
gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates, ps.g_mean_psf_mag, ps.i_mean_psf_mag
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(gaia.ra, gaia.dec), 
                   POLYGON(135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))
  AND 1 = CONTAINS(POINT(gaia.pmra, gaia.pmdec),
                   POLYGON( -4.05037121,-14.75623261, -3.41981085,-14.72365546, -3.03521988,-14.44357135, -2.26847919,-13.7140236 , -2.61172203,-13.24797471, -2.73471401,-13.09054471, -3.19923146,-12.5942653 , -3.34082546,-12.47611926, -5.67489413,-11.16083338, -5.95159272,-11.10547884, -6.42394023,-11.05981295, -7.09631023,-11.95187806, -7.30641519,-12.24559977, -7.

In [36]:
candidate_df = make_dataframe(candidate_table)

In [37]:
filename = 'gd1_data.hdf'

candidate_df.to_hdf(filename, 'candidate_df')

In [38]:
candidate_df.to_csv('gd1_data.csv')

# Episode 6: HDF5

In [39]:
from matplotlib.patches import Polygon

In [40]:
filename = '../student_download/gd1_isochrone.hdf5'
iso_df = pd.read_hdf(filename, 'iso_df')

In [41]:
g_all = iso_df['mag_g']

g_mask = (g_all > 18.0) & (g_all < 21.5)

In [42]:
iso_masked = iso_df[g_mask]

In [43]:
g = iso_masked['mag_g']
left_color = iso_masked['color_g_i'] - 0.06
right_color = iso_masked['color_g_i'] + 0.12

In [44]:
color_loop = front_to_back(left_color, right_color)

In [45]:
mag_loop = front_to_back(g, g)

In [46]:
loop_df = pd.DataFrame()
loop_df['color_loop'] = color_loop
loop_df['mag_loop'] = mag_loop

In [47]:
polygon = Polygon(loop_df)

In [48]:
filename = 'gd1_data.hdf'
loop_df.to_hdf(filename, 'loop_df')

In [49]:
cmd_df = pd.DataFrame()

cmd_df['color'] = candidate_df['g_mean_psf_mag'] - candidate_df['i_mean_psf_mag']
cmd_df['mag'] = candidate_df['g_mean_psf_mag']

In [50]:
inside_mask = polygon.contains_points(cmd_df)

In [51]:
winner_df = candidate_df[inside_mask]

In [52]:
filename = 'gd1_data.hdf'
winner_df.to_hdf(filename, 'winner_df')

# Move Data Files to Download Directory

In [53]:
mv gd1_results.fits ../student_download/data/

In [54]:
mv gd1_data.csv ../student_download/data/

In [55]:
mv gd1_data.hdf ../student_download/data/